In [1]:
import pandas as pd

# Cargar los archivos TSV
current_file = 'rules-2025.05.06-13.03.tsv'
previous_file = 'rules-2025.04.01-13.31.tsv'

df_current = pd.read_csv(current_file, sep='\t', encoding='latin-1', on_bad_lines='skip')
df_previous = pd.read_csv(previous_file, sep='\t', encoding='latin-1', on_bad_lines='skip')

# Contar different_topics en el dataframe original
original_topics_current = df_current['Topic'].nunique()
original_topics_previous = df_previous['Topic'].nunique()

# Contar IDs totales y clasificados por estado 'Active' para el dataframe original df[df['Active'] == True]['ID'].nunique()
current_total_ids = df_current['ID'].nunique()
current_active_intents = df_current[df_current['Active'] == True]['ID'].nunique()
current_inactive_intents = df_current[df_current['Active'] == False]['ID'].nunique()
current_active_topics = df_current[df_current['Active'] == True]['Topic'].nunique()
current_inactive_topics = original_topics_current - current_active_topics

previous_total_ids = df_previous['ID'].nunique()
previous_active_intents = df_previous[df_previous['Active'] == True]['ID'].nunique()
previous_inactive_intents = df_previous[df_previous['Active'] == False]['ID'].nunique()
previous_active_topics = df_previous[df_previous['Active'] == True]['Topic'].nunique()
previous_inactive_topics = original_topics_previous - previous_active_topics

# Filtrar contenidos apagados y Topic path que comiencen con los strings especificados
excluded_topics = [
    'Core de bot', 'Revisar', '2. Onboardings', 'Derivaciones', 'Push', 'Notificaciones', 
    'Z. Apagados, detonables', 'Z. Apagados, reciclables', 'Test', 'Utilidades para intenciones', 
    'AWS Polly', 'Prueba',' '
]

# Filtrar valores vacíos en 'Topic path' 
# Filtrar intenciones que contienen Login y Push
def filter_df(df):
    filtered = df[~((df['Active'] == False) | (df['Topic path'].str.startswith(tuple(excluded_topics))))]
    filtered = filtered[filtered['Topic path'].notna()]
    filtered = filtered[~((filtered['Topic path'].str.contains('Login')) | (filtered['Name'].str.contains('push|PUSH|Push')))]
    return filtered

filtered_current = filter_df(df_current)
filtered_previous = filter_df(df_previous)

# Contar intenciones relevantes en los dataframes filtrados
relevant_intentions_current = filtered_current['ID'].nunique()
relevant_intentions_previous = filtered_previous['ID'].nunique()

# Contar contenidos relevantes en los dataframes filtrados
relevant_topic_current = filtered_current['Topic'].nunique()
relevant_topic_previous = filtered_previous['Topic'].nunique()

# Clasificar según la longitud de 'Bot Says'
filtered_current['Classification'] = filtered_current['Bot Says'].apply(lambda x: 'action' if pd.isna(x) or len(x) <= 1 else 'wording')
filtered_previous['Classification'] = filtered_previous['Bot Says'].apply(lambda x: 'action' if pd.isna(x) or len(x) <= 1 else 'wording')

# Contar IDs para cada clasificación
actions_int_current = filtered_current[filtered_current['Classification'] == 'action']['ID'].nunique()
wording_int_current = filtered_current[filtered_current['Classification'] == 'wording']['ID'].nunique()
# Contar Topic únicos para cada clasificación
actions_topic_current = filtered_current[filtered_current['Classification'] == 'action']['Topic'].nunique()
wording_topic_current = filtered_current[filtered_current['Classification'] == 'wording']['Topic'].nunique()


actions_int_previous = filtered_previous[filtered_previous['Classification'] == 'action']['ID'].nunique()
wording_int_previous = filtered_previous[filtered_previous['Classification'] == 'wording']['ID'].nunique()
actions_topic_previous = filtered_previous[filtered_previous['Classification'] == 'action']['Topic'].nunique()
wording_topic_previous = filtered_previous[filtered_previous['Classification'] == 'wording']['Topic'].nunique()

# Comparar los dataframes filtrados
current_ids = set(filtered_current['ID'])
previous_ids = set(filtered_previous['ID'])

# Nuevas, eliminadas y modificadas
new_ids = current_ids - previous_ids
removed_ids = previous_ids - current_ids
common_ids = current_ids & previous_ids

modified_ids = [
    id_ for id_ in common_ids
    if not filtered_current.loc[filtered_current['ID'] == id_, 'Bot Says'].equals(
        filtered_previous.loc[filtered_previous['ID'] == id_, 'Bot Says']
    ) and abs(
        len(" ".join(
            filtered_current.loc[filtered_current['ID'] == id_, 'Bot Says'].dropna().astype(str).tolist()
        )) -
        len(" ".join(
            filtered_previous.loc[filtered_previous['ID'] == id_, 'Bot Says'].dropna().astype(str).tolist()
        ))
    ) > 5
]

# Contar new, removed y modified different_topics
current_topics_set = set(filtered_current['Topic'])
previous_topics_set = set(filtered_previous['Topic'])

new_topics = current_topics_set - previous_topics_set
removed_topics = previous_topics_set - current_topics_set
# Identify topics with differing "Bot Says" values and differences > 10 characters
modified_topics = [
    topic for topic in current_topics_set & previous_topics_set
    if not filtered_current.loc[
        filtered_current['Topic'] == topic, 'Bot Says'
    ].equals(
        filtered_previous.loc[
            filtered_previous['Topic'] == topic, 'Bot Says'
        ]
    ) and abs(
        len(" ".join(
            str(value) for value in filtered_current.loc[
                filtered_current['Topic'] == topic, 'Bot Says'
            ].dropna().tolist()
        )) -
        len(" ".join(
            str(value) for value in filtered_previous.loc[
                filtered_previous['Topic'] == topic, 'Bot Says'
            ].dropna().tolist()
        ))
    ) > 5
]
# Imprimir resultados
print(f"Mes Actual - Total Intenciones en el Bot: {current_total_ids}")
print(f"Mes Actual - Total Contenidos en el Bot: {original_topics_current}")
print(f"Mes Actual - Intenciones Prendidas: {current_active_intents}")
print(f"Mes Actual - Intenciones Apagadas: {current_inactive_intents}")
print(f"Mes Actual - Contenidos Prendidos: {current_active_topics}")
print(f"Mes Actual - Contenidos Apagados: {current_inactive_topics}")

print(f"Mes Anterior - Total Intenciones en el Bot: {previous_total_ids}")
print(f"Mes Anterior - Total Contenidos en el Bot: {original_topics_previous}")
print(f"Mes Anterior - Intenciones Prendidas: {previous_active_intents}")
print(f"Mes Anterior - Intenciones Apagadas: {previous_inactive_intents}")
print(f"Mes Anterior - Contenidos Prendidos: {previous_active_topics}")
print(f"Mes Anterior - Contenidos Apagados: {previous_inactive_topics}")

print(f"Mes Actual - Total Intenciones Relevantes en el Bot: {relevant_intentions_current}")
print(f"Mes Actual - Total Contenidos Relevantes en el Bot: {relevant_topic_current}")

print(f"Mes Anterior - Total Intenciones Relevantes en el Bot: {relevant_intentions_previous}")
print(f"Mes Anterior - Total Contenidos Relevantes en el Bot: {relevant_topic_previous}")


print(f"Mes Actual - Total Intenciones Relevantes de Wording en el Bot: {wording_int_current}")
print(f"Mes Actual - Total Intenciones Relevantes de Acciones en el Bot: {actions_int_current}")
print(f"Mes Actual - Total Contenidos Relevantes de Wording en el Bot: {wording_topic_current}")
print(f"Mes Actual - Total Contenidos Relevantes de Acciones en el Bot: {actions_topic_current}")

print(f"Mes Anterior - Total Intenciones Relevantes de Wording en el Bot: {wording_int_previous}")
print(f"Mes Anterior - Total Intenciones Relevantes de Acciones en el Bot: {actions_int_previous}")
print(f"Mes Anterior - Total Contenidos Relevantes de Wording en el Bot: {wording_topic_previous}")
print(f"Mes Anterior - Total Contenidos Relevantes de Acciones en el Bot: {actions_topic_previous}")

# Crear un diccionario con los datos
data = {
    "Dato": [
        "Total Intenciones en el Bot",
        "Total Contenidos en el Bot",
        "Intenciones Prendidas",
        "Intenciones Apagadas",
        "Contenidos Prendidos",
        "Contenidos Apagados",
        "Total Intenciones Relevantes en el Bot",
        "Total Contenidos Relevantes en el Bot",
        "Total Intenciones Relevantes de Wording en el Bot",
        "Total Intenciones Relevantes de Acciones en el Bot",
        "Total Contenidos Relevantes de Wording en el Bot",
        "Total Contenidos Relevantes de Acciones en el Bot"
    ],
    "Mes Actual": [
        current_total_ids,
        original_topics_current,
        current_active_intents,
        current_inactive_intents,
        current_active_topics,
        current_inactive_topics,
        relevant_intentions_current,
        relevant_topic_current,
        wording_int_current,
        actions_int_current,
        wording_topic_current,
        actions_topic_current
    ],
    "Mes Anterior": [
        previous_total_ids,
        original_topics_previous,
        previous_active_intents,
        previous_inactive_intents,
        previous_active_topics,
        previous_inactive_topics,
        relevant_intentions_previous,
        relevant_topic_previous,
        wording_int_previous,
        actions_int_previous,
        wording_topic_previous,
        actions_topic_previous
    ]
}

# Crear un DataFrame
dfx = pd.DataFrame(data)

# Guardar el DataFrame en un archivo Excel
dfx.to_excel("datos_contenido.xlsx", index=False)


Mes Actual - Total Intenciones en el Bot: 11549
Mes Actual - Total Contenidos en el Bot: 801
Mes Actual - Intenciones Prendidas: 9043
Mes Actual - Intenciones Apagadas: 2506
Mes Actual - Contenidos Prendidos: 686
Mes Actual - Contenidos Apagados: 115
Mes Anterior - Total Intenciones en el Bot: 11325
Mes Anterior - Total Contenidos en el Bot: 794
Mes Anterior - Intenciones Prendidas: 8910
Mes Anterior - Intenciones Apagadas: 2415
Mes Anterior - Contenidos Prendidos: 680
Mes Anterior - Contenidos Apagados: 114
Mes Actual - Total Intenciones Relevantes en el Bot: 7497
Mes Actual - Total Contenidos Relevantes en el Bot: 601
Mes Anterior - Total Intenciones Relevantes en el Bot: 7418
Mes Anterior - Total Contenidos Relevantes en el Bot: 595
Mes Actual - Total Intenciones Relevantes de Wording en el Bot: 6564
Mes Actual - Total Intenciones Relevantes de Acciones en el Bot: 933
Mes Actual - Total Contenidos Relevantes de Wording en el Bot: 586
Mes Actual - Total Contenidos Relevantes de Accio

In [2]:

print(f"Intenciones Nuevas: {len(new_ids)}")
print(f"Intenciones Eliminadas: {len(removed_ids)}")
print(f"Intenciones Modificadas: {len(modified_ids)}")

print(f"Topics Nuevos: {len(new_topics)}")
print(f"Topics Eliminados: {len(removed_topics)}")
print(f"Topics Modificados: {len(modified_topics)}")

Intenciones Nuevas: 158
Intenciones Eliminadas: 79
Intenciones Modificadas: 167
Topics Nuevos: 10
Topics Eliminados: 4
Topics Modificados: 68


##Se suma esto para un análisis interanual 

In [3]:
# Identify unmodified topics -2024
unmodified_topics = current_topics_set & previous_topics_set - set(modified_topics)

# Save unmodified topics to an Excel file
unmodified_df = pd.DataFrame(unmodified_topics, columns=['Unmodified Topics'])
unmodified_df.to_excel('unmodified_topics.xlsx', index=False)

print("Contenidos no modificados han sido guardados en  unmodified_topics.xlsx")

Contenidos no modificados han sido guardados en  unmodified_topics.xlsx
